In [59]:
with open("lucas.txt", "r", encoding="utf-8", errors="ignore") as file:
    text = file.read()


In [60]:
chars = sorted(list(set(text)))
vocab = len(chars)
print("".join(chars))
print(vocab)

	
 !"$%&'()+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz~‘’“”
86


In [61]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])


In [62]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:1000])

torch.Size([211199]) <built-in method type of Tensor object at 0x000001F36C21B390>
tensor([ 0, 36, 55, 70, 70, 79,  2, 36, 55, 66, 66, 69, 77, 59, 59, 68,  3,  2,
        37,  2, 57, 55, 68, 83, 74,  2, 56, 59, 66, 63, 59, 76, 59,  2, 63, 74,
        83, 73,  2, 42, 69, 76, 59, 67, 56, 59, 72,  2, 55, 66, 72, 59, 55, 58,
        79,  3,  2, 48, 62, 63, 73,  2, 79, 59, 55, 72,  2, 63, 73,  2, 67, 79,
         2, 60, 63, 72, 73, 74,  2, 79, 59, 55, 72,  2, 68, 69, 74,  2, 61, 69,
        63, 68, 61,  2, 74, 72, 63, 57, 65, 13, 69, 72, 13, 74, 72, 59, 55, 74,
        63, 68, 61, 12,  2, 77, 62, 63, 57, 62,  2, 73, 63, 61, 68, 63, 60, 63,
        57, 55, 68, 74, 66, 79,  2, 72, 59, 58, 75, 57, 59, 58,  2, 67, 79,  2,
        63, 68, 57, 69, 67, 59,  2, 63, 68,  2, 74, 59, 72, 67, 73,  2, 69, 60,
         2, 57, 62, 69, 57, 69, 66, 55, 74, 59,  2, 55, 68, 58,  2, 69, 74, 62,
        59, 72,  2, 57, 55, 68, 58, 63, 59, 73, 14,  2, 36, 69, 77, 59, 76, 59,
        72, 12,  2, 37,  2, 73, 74, 6

In [63]:
n = (int)(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [64]:
block_size = 8
train_data[:block_size + 1]

tensor([ 0, 36, 55, 70, 70, 79,  2, 36, 55])

In [65]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # finds 4 random starting indices for getting blocks
    x = torch.stack([data[i:i+block_size] for i in ix]) # gets a stack of the 4 blocks of 8 values
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # gets the next value (which is what we're trying to predict)
    return x, y

xb, yb = get_batch("train")

print(xb)
print(yb)

tensor([[56, 59,  2, 55, 56, 66, 59,  2],
        [74,  2, 74, 62, 63, 68, 65,  2],
        [69, 70, 70, 63, 68, 61,  2, 63],
        [73, 69,  2, 60, 55, 72,  2, 55]])
tensor([[59,  2, 55, 56, 66, 59,  2, 74],
        [ 2, 74, 62, 63, 68, 65,  2, 77],
        [70, 70, 63, 68, 61,  2, 63, 68],
        [69,  2, 60, 55, 72,  2, 55, 77]])


In [66]:
import torch
import torch.nn as nn
from torch.nn import functional as F 
torch.manual_seed(1337)

# bigram is a pair of consecutive words in a sequence
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, inputs, targets=None):
        logits = self.token_embedding_table(inputs) # (Batch, Time, Channel)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, inputs, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(inputs)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            inputs_next = torch.multinomial(probs, num_samples=1) # (B, 1) as there is one prediction for what comes next
            inputs = torch.cat((inputs, inputs_next), dim=1)
        
        return inputs

m = BigramLanguageModel(vocab)
logits, loss = m(xb, yb) # passing inputs and targets
print(logits.shape)
print(loss)

inputs = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(inputs, max_new_tokens=100)[0].tolist()))

torch.Size([32, 86])
tensor(4.9079, grad_fn=<NllLossBackward0>)
	F0IBrNR~zu"H;goI	7ufMOi%0w’w
rpdoJNpf“uS5	Z5%sjWABSAwpsJ“’e “
gt9507K;~9k5XN.M(ir’?T&Il”9AMAHY,PL”AT


In [67]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [68]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# print(loss.item())

In [69]:
inputs = torch.zeros((1, 1), dtype=torch.long)
# print(decode(m.generate(inputs, max_new_tokens=1000)[0].tolist()))

In [70]:
xbow = torch.zeros((B, T, C)) # bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)


In [71]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

#a single head of self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

print(wei)

NameError: name 'value' is not defined

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


'device = "cuda" if torch.cuda.is_available() else "cpu"\n\ncontext = torch.zeros((1, 1), dtype=torch.long, device=device)\nprint(decode(m.generate(context, max_new_tokens=500)[0].tolist()))\n'

In [ ]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

k.var()

tensor(0.9487)